Based on the baseline "Radek's LGBMRanker starter-pack”: https://www.kaggle.com/code/marcogorelli/radek-s-lgbmranker-starter-pack


# Helper functions

In [1]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []

    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [3]:
def precision(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]

    correct_predictions = [p for p in predicted if p in actual]

    return len(correct_predictions) / len(predicted)

def recall(actual, predicted, k=12):
    if len(predicted) > k:
        predicted = predicted[:k]

    correct_predictions = [p for p in predicted if p in actual]

    return len(correct_predictions) / len(actual)

def precision12(actual, predicted, k=12):
    return np.mean([precision(a,p,k) for a,p in zip(actual, predicted)])

def recall12(actual, predicted, k=12):
    return np.mean([recall(a,p,k) for a,p in zip(actual, predicted)])

In [4]:
import pandas as pd

# Read input files

In [5]:
%%time
pad = "/kaggle/input/makeparquet"
transactions = pd.read_parquet(pad+'/transactions_train.parquet')
customers = pd.read_parquet(pad+'/customers.parquet')
articles = pd.read_parquet(pad+'/articles.parquet')

CPU times: user 2.16 s, sys: 2.28 s, total: 4.44 s
Wall time: 4.74 s


# Add seasonal boosted feature

In [6]:
import pandas as pd

# Define a function to apply to each row to calculate the new column value
def color_or_fabric(row):
    if row['colour_group_code'] in [7, 43] or row['fabric_wool'] == 1:
        return 1
    else:
        return 0

# Apply the function to create the new column 'color_or_fabric'
articles['autumnBoosted'] = articles.apply(color_or_fabric, axis=1)

# Set Validation and prune transactions on date

In [7]:
val = transactions[transactions.week == transactions.week.max()]

Comment one of the blocks out whether you want a test run or a sumbission run

In [8]:
Testing = True
test_week = transactions.week.max()
transactions = transactions[transactions.week > transactions.week.max() - 10]
transactions = transactions[transactions.week < transactions.week.max()]

In [9]:
# Testing = False
# test_week = transactions.week.max()+1
# transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [10]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 21.1 s, sys: 203 ms, total: 21.3 s
Wall time: 21.3 s


In [11]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [12]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 547 ms, sys: 17.7 ms, total: 565 ms
Wall time: 564 ms


In [13]:
candidates_last_purchase = transactions.copy()

In [14]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])

candidates_last_purchase.week=weeks

CPU times: user 16.3 s, sys: 90 ms, total: 16.4 s
Wall time: 16.3 s


### Bestsellers candidates

In [15]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [16]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [17]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [18]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [19]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [20]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,season,year,month,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,2,-28,7,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,2,-28,7,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,2,-28,7,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,2,-28,7,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,2,-28,7,95
...,...,...,...,...,...,...,...,...,...
31521960,2020-09-15,18439897732908966680,794321007,0.061000,2,3,-28,9,103
31531712,2020-09-15,18444276791873187543,867969008,0.033881,2,3,-28,9,103
31539937,2020-09-15,18444799607866739422,909721003,0.042356,2,3,-28,9,103
31543799,2020-09-15,18446250046654386343,869872006,0.033881,1,3,-28,9,103


In [21]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [22]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [23]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [24]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [25]:
transactions['purchased'] = 1

In [26]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [27]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [28]:
data.purchased.mean()

0.13579181084519196

### Add bestseller information

In [29]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [30]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [31]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [32]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [33]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [34]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [35]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank',
'buys_womens_clothing', 'buys_mens_clothes', 'buys_kid_baby_clothes', 'season',
'fabric_jeans','fabric_cotton', 'fabric_wool', 'fabric_polyester', 'fabric_silk', 'fabric_denim', 'fabric_linen', 'fabric_spandex', 'fabric_rayon', 'fabric_nylon', 'fabric_leather', 'fabric_suede',
'autumnBoosted']
#  'year', 'month',

In [36]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 393 ms, sys: 189 ms, total: 582 ms
Wall time: 579 ms


# Model training

In [37]:
from lightgbm.sklearn import LGBMRanker

In [38]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

# Boost seasonal feature

In [39]:
sample_weights = [2 if autumnBoosted == 1 else 1 for autumnBoosted in train_X['autumnBoosted']]


In [40]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
    sample_weight=sample_weights  # Use sample weights
)

[LightGBM] [Info] Loading query weights...
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.905733
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.542235
[LightGBM] [Debug] init for col-wise cost 0.959954 seconds, init for row-wise cost 2.303538 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.545566 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1109
[LightGBM] [Info] Number of data points in the train set: 10205052, number of used features: 34
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
CPU times: user 29 s, sys: 2.3 s, total: 31.3 s
Wall time: 12.5 s


In [41]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9962709464663856
season 0.00254325057488934
buys_kid_baby_clothes 0.00045367677222126694
age 0.00022416996550489576
buys_mens_clothes 0.00015890162233198903
fabric_wool 6.42673572945795e-05
product_type_no 4.831104818682237e-05
club_member_status 4.714161441205282e-05
postal_code 3.8716484374915604e-05
perceived_colour_value_id 3.8267316518420724e-05
article_id 3.335042368171536e-05
garment_group_no 2.6965350561717527e-05
Active 1.8510743441764202e-05
graphical_appearance_no 1.2813621122884336e-05
fashion_news_frequency 1.1690443880555831e-05
index_group_no 9.020195191537174e-06
fabric_cotton 0.0
fabric_leather 0.0
colour_group_code 0.0
fabric_nylon 0.0
perceived_colour_master_id 0.0
department_no 0.0
index_code 0.0
section_no 0.0
FN 0.0
fabric_jeans 0.0
fabric_rayon 0.0
fabric_spandex 0.0
fabric_linen 0.0
fabric_suede 0.0
buys_womens_clothing 0.0
fabric_denim 0.0
fabric_silk 0.0
fabric_polyester 0.0
autumnBoosted 0.0


# Calculate predictions

In [42]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


# Create submission

In [43]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [44]:
%%time
preds = []
predsval = []
positive_items_val = val.groupby(['customer_id'])['article_id'].apply(list)
# creating validation set for metrics
val_users = positive_items_val.keys()
val_items = []
# Go over all customers and get a prediction
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])
#     if the customer bought something in the validation set add it to the validation list
    if c_id in val_users:
        predsval.append(pred[:12])
        val_items.append(positive_items_val[c_id])

CPU times: user 8.85 s, sys: 169 ms, total: 9.02 s
Wall time: 9.02 s


In [45]:
if Testing:
    print("Total users in validation:", len(val_users))
    print("mAP12 Score on Validation set:", mapk(val_items, predsval))
    print("precision Score on Validation set:", precision12(val_items, predsval))
    print("recall Score on Validation set:", recall12(val_items, predsval))

Total users in validation: 68984
mAP12 Score on Validation set: 0.022771612247049183
precision Score on Validation set: 0.010159360624685917
recall Score on Validation set: 0.0477448074224207


In [46]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [47]:
sub_name = 'SeasonBoosted_model_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)